## CONNECT TO ELASTICSEARCH

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "So0xTl9tCAdXqx4JniGO"),
    ca_certs="http_ca.crt",
)

client_info = es.info()

print("connected to elasticsearch")
pprint(client_info.body)


connected to elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'Ptp3XLACR3-Wmx0PcdxoZA',
 'name': 'c899a61c3425',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-08-24T22:05:04.526302670Z',
             'build_flavor': 'default',
             'build_hash': '0c781091a2f57de895a73a1391ff8426c0153c8d',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.2.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.1.3'}}


### common types

#### 1.1 binary

In [2]:
es.indices.create(
    index="binary_index",
    mappings={
        "properties":{
            "image_data": {
                "type": "binary"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'binary_index'})

In [4]:
import base64

image_path = './images/field_data_types_docs.png'

with open(image_path, 'rb') as image_file:
    image_bytes = image_file.read()
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')

print(image_base64[:100])
print(len(image_base64))


iVBORw0KGgoAAAANSUhEUgAAB4AAAAJTCAYAAADpMAvgAAAABHNCSVQICAgIfAhkiAAAABl0RVh0U29mdHdhcmUAZ25vbWUtc2Ny
271328


In [5]:
document = {
    'image_data': image_base64
}
response = es.index(index="binary_index", body=document)
response

ObjectApiResponse({'_index': 'binary_index', '_id': 'KZVPTJkBza3mxyTRqKVw', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

##### 1.2 others

In [6]:
es.indices.create(
    index="other_common_data_types_index",
    mappings={
        'properties':{
            'book_reference': {
                'type': 'keyword'
            },
            'price': {
                'type': 'float'
            },
            'publish_date':{
                'type': 'date'
            },
            'is_available':{
                'type': 'boolean'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'other_common_data_types_index'})

In [7]:
document = {
    'book_reference': 'ISBN-123-456-789',
    'price': 19.99,
    'publish_date': '2023-10-01',
    'is_available': True
}
response = es.index(index="other_common_data_types_index", body=document)
response

ObjectApiResponse({'_index': 'other_common_data_types_index', '_id': 'KpVTTJkBza3mxyTRDqWB', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

#### 2. Object types

##### 2.1 object

In [8]:
es.indices.create(
    index="object_index",
    mappings={
        'properties':{
            'author': {
                'properties':{
                    'first_name': {'type': 'text'},
                    'last_name': {'type': 'text'}
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'object_index'})

In [9]:
document = {
    'author': {
        'first_name': 'ahmad',
        'last_name': 'alnujaidi'
    }
}
response = es.index(index="object_index", body=document)
response

ObjectApiResponse({'_index': 'object_index', '_id': 'K5VbTJkBza3mxyTRU6VN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

##### 2.2 flattened objects

In [10]:
es.indices.create(
    index="flattened_object_index",
    mappings={
        'properties':{
            'author':{
                'type': 'flattened'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'flattened_object_index'})

In [12]:
document = {
    'author': {
        'first_name': 'ibrahim',
        'last_name': 'nujaidi'
    }
}
response = es.index(index="flattened_object_index", body=document)
response

ObjectApiResponse({'_index': 'flattened_object_index', '_id': 'LJXCTJkBza3mxyTRaKWX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

#### 2.3 Nested object

In [13]:
es.indices.create(
    index="nested_object_index",
    mappings={
        'properties':{
            'user':{
                'type':'nested'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'nested_object_index'})

In [14]:
documents = [
    {
        'first': "john",
        'last':'doe'
    },
    {
        'first': "yousif",
        'last':'smith'
    }
]

response = es.index(index="nested_object_index", body={'user': documents})
response

ObjectApiResponse({'_index': 'nested_object_index', '_id': 'LZXETJkBza3mxyTR-6Vk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### Text search type

#### 3.1 Text

In [15]:
es.indices.create(
    index='text_index',
    mappings={
        'properties':{
            'email_body':{
                'type':'text'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_index'})

In [16]:
document = {
    'email_body': 'Hello this is ahmad writing a test email'
}
response = es.index(index="text_index", body=document)
response

ObjectApiResponse({'_index': 'text_index', '_id': 'MpXSTJkBza3mxyTRBKUA', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

#### 3.2 Completion

In [17]:
es.indices.create(
    index="text_completion_index",
    mappings={
        'properties':{
            'suggest':{
                'type':'completion'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_completion_index'})

In [19]:
document = {
    'suggest': {
        'input': ["Mars", "Planet"]
    }
}
document_2={
    'suggest':{
        'input': ['Andromeda', 'Galaxy']
    }
}
response = es.index(index="text_completion_index", body=document)
response_2 = es.index(index="text_completion_index", body=document_2)

response

ObjectApiResponse({'_index': 'text_completion_index', '_id': 'ZJXWTJkBza3mxyTRXKX9', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

### 4. Spatial Data types

#### 4.1 Geo point

In [20]:
es.indices.create(
    index="geo_point_index",
    mappings={
        'properties':{
            'location':{
                'type':'geo_point'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_point_index'})

In [21]:
document = {
    'location':{
        'type':'Point',
        'coordinates': [40.7128, -74.0060]
    }
}
response = es.index(index="geo_point_index", body=document)
response

ObjectApiResponse({'_index': 'geo_point_index', '_id': 'ZpXbTJkBza3mxyTRvKW0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

#### 4.2 Geo shape

In [22]:
es.indices.create(
    index="geo_shape_index",
    mappings={
        'properties':{
            'location':{
                'type':'geo_shape'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_shape_index'})

In [23]:
document_1 ={
    'location':{
        'type': 'LineString',
        'coordinates': [
            [-77.03633, 38.89511],
            [-77.00900, 38.88993]
        ]
    }
}

document_2 ={
    'location':{
        'type': 'Polygon',
        'coordinates': [
            [
                [100, 0],
                [101, 0],
                [101, 1],
                [100, 1],
                [100, 0]
            ],
            [
                [100.2, 0.2],
                [100.8, 0.2],
                [100.8, 0.8],
                [100.2, 0.8],
                [100.2, 0.2]
            ]
        ]
    }
}

es.index(index="geo_shape_index", body=document_1)
es.index(index="geo_shape_index", body=document_2)

ObjectApiResponse({'_index': 'geo_shape_index', '_id': 'aJXhTJkBza3mxyTR86XP', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

#### 4.3 Point

In [35]:
es.indices.delete(index='point_index', ignore_unavailable=True)
es.indices.create(
    index='point_index',
    mappings={
        'properties':{
            'location':{
                'type':'point'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'point_index'})

In [ ]:
document = {
    'location': {
        'coordinates': [
            -74.00,
            40.71 
            
        ]
    }
}
# response = es.index(index='point_index', body=document)

BadRequestError: BadRequestError(400, 'document_parsing_exception', 'Required [type]', Required [type])